In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import itertools
import gc

In [63]:
features = pd.read_csv('../data/nbme-score-clinical-patient-notes/features.csv')
features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
features

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
...,...,...,...
138,912,9,Family-history-of-migraines
139,913,9,Female
140,914,9,Photophobia
141,915,9,No-known-illness-contacts


In [64]:
pn = pd.read_csv('../data/nbme-score-clinical-patient-notes/patient_notes.csv')
pn

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...
...,...,...,...
42141,95330,9,Ms. Madden is a 20 yo female presenting w/ the...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...
42144,95333,9,Stephanie madden is a 20 year old woman compla...


In [65]:
pn_fea = pn.merge(features, on='case_num', how='left')
pn_fea

,pn_num,case_num,pn_history,feature_num,feature_text
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded
...,...,...,...,...,...
626897,95334,9,patient is a 20 yo F who presents with a heada...,912,Family-history-of-migraines
626898,95334,9,patient is a 20 yo F who presents with a heada...,913,Female
626899,95334,9,patient is a 20 yo F who presents with a heada...,914,Photophobia
626900,95334,9,patient is a 20 yo F who presents with a heada...,915,No-known-illness-contacts


In [66]:
train = pd.read_pickle('../data/train_processed.pkl')
train

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history,annotation_length,fold
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...,1,4
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...,1,4
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...,1,4
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...,2,4
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...,1,4
...,...,...,...,...,...,...,...,...,...,...
14295,95333_912,9,95333,912,[],[],Family-history-of-migraines,Stephanie madden is a 20 year old woman compla...,0,4
14296,95333_913,9,95333,913,[],[],Female,Stephanie madden is a 20 year old woman compla...,0,4
14297,95333_914,9,95333,914,[photobia],[274 282],Photophobia,Stephanie madden is a 20 year old woman compla...,1,4
14298,95333_915,9,95333,915,[no sick contacts],[421 437],No-known-illness-contacts,Stephanie madden is a 20 year old woman compla...,1,4


In [67]:
for pnn in tqdm(train.pn_num.unique()):
    pn_fea = pn_fea.drop(pn_fea[pn_fea['pn_num']==pnn].index)
pn_fea = pn_fea.reset_index(drop=True)
pn_fea

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:22<00:00, 44.05it/s]


,pn_num,case_num,pn_history,feature_num,feature_text
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded
...,...,...,...,...,...
612597,95334,9,patient is a 20 yo F who presents with a heada...,912,Family-history-of-migraines
612598,95334,9,patient is a 20 yo F who presents with a heada...,913,Female
612599,95334,9,patient is a 20 yo F who presents with a heada...,914,Photophobia
612600,95334,9,patient is a 20 yo F who presents with a heada...,915,No-known-illness-contacts


In [68]:
from sklearn.model_selection import StratifiedKFold
Fold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
pn_fea.loc[:, 'fold'] = -1
for n, (train_index, val_index) in enumerate(Fold.split(pn_fea, pn_fea.case_num)):
    pn_fea.loc[val_index, 'fold'] = int(n)
pn_fea

,pn_num,case_num,pn_history,feature_num,feature_text,fold
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...,4
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder,2
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure,3
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms,4
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded,0
...,...,...,...,...,...,...
612597,95334,9,patient is a 20 yo F who presents with a heada...,912,Family-history-of-migraines,3
612598,95334,9,patient is a 20 yo F who presents with a heada...,913,Female,2
612599,95334,9,patient is a 20 yo F who presents with a heada...,914,Photophobia,2
612600,95334,9,patient is a 20 yo F who presents with a heada...,915,No-known-illness-contacts,0


In [69]:
def n2s(n, l):
    sn = str(n)
    return '0'*(l-len(sn))+sn

n2s(3,3), n2s(3,5)

('003', '00003')

In [70]:
pn_fea['id'] = pn_fea['pn_num'].apply(lambda x: n2s(x, 5)) + '_' + pn_fea['feature_num'].apply(lambda x: n2s(x, 3))
pn_fea

,pn_num,case_num,pn_history,feature_num,feature_text,fold,id
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...,4,00000_000
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder,2,00000_001
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure,3,00000_002
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms,4,00000_003
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded,0,00000_004
...,...,...,...,...,...,...,...
612597,95334,9,patient is a 20 yo F who presents with a heada...,912,Family-history-of-migraines,3,95334_912
612598,95334,9,patient is a 20 yo F who presents with a heada...,913,Female,2,95334_913
612599,95334,9,patient is a 20 yo F who presents with a heada...,914,Photophobia,2,95334_914
612600,95334,9,patient is a 20 yo F who presents with a heada...,915,No-known-illness-contacts,0,95334_915


In [72]:
pn_fea.to_pickle('../data/train_pl_all.pkl')

In [3]:
train = pd.read_pickle('../data/train_processed.pkl')
train.columns

Index(['id', 'case_num', 'pn_num', 'feature_num', 'annotation', 'location',
       'feature_text', 'pn_history', 'annotation_length', 'fold'],
      dtype='object')

In [6]:
train['char_logits'] = train['pn_history'].apply(lambda x: np.zeros(len(x)))
train

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history,annotation_length,fold,char_logits
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...,1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...,1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...,1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...,2,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...,1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
14295,95333_912,9,95333,912,[],[],Family-history-of-migraines,Stephanie madden is a 20 year old woman compla...,0,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
14296,95333_913,9,95333,913,[],[],Female,Stephanie madden is a 20 year old woman compla...,0,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
14297,95333_914,9,95333,914,[photobia],[274 282],Photophobia,Stephanie madden is a 20 year old woman compla...,1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
14298,95333_915,9,95333,915,[no sick contacts],[421 437],No-known-illness-contacts,Stephanie madden is a 20 year old woman compla...,1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
pl = pd.read_pickle('../data/train_pl_all.pkl')
pl

,pn_num,case_num,pn_history,feature_num,feature_text,fold,id
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...,4,00000_000
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder,2,00000_001
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure,3,00000_002
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms,4,00000_003
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded,0,00000_004
...,...,...,...,...,...,...,...
612597,95334,9,patient is a 20 yo F who presents with a heada...,912,Family-history-of-migraines,3,95334_912
612598,95334,9,patient is a 20 yo F who presents with a heada...,913,Female,2,95334_913
612599,95334,9,patient is a 20 yo F who presents with a heada...,914,Photophobia,2,95334_914
612600,95334,9,patient is a 20 yo F who presents with a heada...,915,No-known-illness-contacts,0,95334_915


In [16]:
pl_logits = pickle.load(open('../data/pl_logits.pkl', 'rb'))
pl = pl.merge(pd.DataFrame({'id': list(pl_logits.keys()), 'char_logits': list(pl_logits.values())}), on='id', how='left')
del pl_logits
gc.collect()
pl

,pn_num,case_num,pn_history,feature_num,feature_text,fold,id,char_logits
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...,4,00000_000,"[-12.14199161529541, -12.14199161529541, -12.2..."
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder,2,00000_001,"[-12.330534934997559, -12.330534934997559, -12..."
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure,3,00000_002,"[-12.260808944702148, -12.260808944702148, -12..."
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms,4,00000_003,"[-12.0237455368042, -12.0237455368042, -12.144..."
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded,0,00000_004,"[-11.666829109191895, -11.666829109191895, -11..."
...,...,...,...,...,...,...,...,...
612597,95334,9,patient is a 20 yo F who presents with a heada...,912,Family-history-of-migraines,3,95334_912,"[-12.142287254333496, -12.142287254333496, -12..."
612598,95334,9,patient is a 20 yo F who presents with a heada...,913,Female,2,95334_913,"[-12.189107894897461, -12.189107894897461, -12..."
612599,95334,9,patient is a 20 yo F who presents with a heada...,914,Photophobia,2,95334_914,"[-12.225860595703125, -12.225860595703125, -12..."
612600,95334,9,patient is a 20 yo F who presents with a heada...,915,No-known-illness-contacts,0,95334_915,"[-11.919486045837402, -11.919486045837402, -11..."


In [17]:
def get_spans(char_logits, texts, th=0):
    results = []
    for i, char_prob in enumerate(char_logits):
        result = np.where(char_prob > th)[0]
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        temp = []
        for r in result:
            s, e = min(r), max(r)
            while texts[i][s] == ' ':
                s += 1
            while texts[i][e] == ' ':
                e -= 1
            temp.append([s, e+1])
        result = temp
        results.append(result)
    return results

pl['spans'] = get_spans(pl.char_logits.values, pl.pn_history.values)
pl

,pn_num,case_num,pn_history,feature_num,feature_text,fold,id,char_logits,spans
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...,4,00000_000,"[-12.14199161529541, -12.14199161529541, -12.2...","[[553, 566]]"
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder,2,00000_001,"[-12.330534934997559, -12.330534934997559, -12...","[[576, 597]]"
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure,3,00000_002,"[-12.260808944702148, -12.260808944702148, -12...",[]
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms,4,00000_003,"[-12.0237455368042, -12.0237455368042, -12.144...","[[216, 228]]"
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded,0,00000_004,"[-11.666829109191895, -11.666829109191895, -11...",[]
...,...,...,...,...,...,...,...,...,...
612597,95334,9,patient is a 20 yo F who presents with a heada...,912,Family-history-of-migraines,3,95334_912,"[-12.142287254333496, -12.142287254333496, -12...","[[794, 796], [798, 811]]"
612598,95334,9,patient is a 20 yo F who presents with a heada...,913,Female,2,95334_913,"[-12.189107894897461, -12.189107894897461, -12...","[[19, 20]]"
612599,95334,9,patient is a 20 yo F who presents with a heada...,914,Photophobia,2,95334_914,"[-12.225860595703125, -12.225860595703125, -12...","[[486, 504]]"
612600,95334,9,patient is a 20 yo F who presents with a heada...,915,No-known-illness-contacts,0,95334_915,"[-11.919486045837402, -11.919486045837402, -11...",[]


In [22]:
pl.loc[612597, 'pn_history'][796:798]

': '